# 深層学習ノートブック-11 PytorchのNNモジュール
深層学習の仕組みの勉強のためにMLPをスクラッチで実装してきたが、  
これはPytorchのtorch.nnモジュールに実装されている。  
ただ、MLPのクラスがまるごとあるわけではなく、線形変換の層、活性化関数(ReLU)、損失関数など、  
MLPを構成するコンポーネントごとに実装されている。  
ユーザとしてはこれらのコンポーネントを組み合わせて学習モデルを構築していくことになる。  

* torch.nnに様々な深層学習のアーキテクチャを構築するためのクラスと関数が入っている
* 層：NNの基本的な層のblock
    * nn.Linear: 全結合層またはFC(Fully Connected)層ともいう。$XW^T+b$が実装されている。
    * nn.ReLU: ReLU層
* 損失関数：ネットワークのパフォーマンスを評価する
    * nn.MSELoss: 平均二乗誤差
    * nn.CrossEntropy: 交差エントロピー
* モジュール：全てのNNモジュールの基本クラス。カスタムでクラスを作成する時はnn.Moduleを継承する
* 関数：単一の関数として層や損失関数, 活性化関数を提供する。torch.nn.funcbonalに入っていて，通常はFとしてimportする

## nn.Linear
線形変換を行う部分の層を表すクラス。

In [1]:
import torch
# NNモジュールを読み込む
from torch import nn
from torch.nn import functional as F

In [2]:
# 入力の次元と出力の次元（隠れ層１層目ならば入力は特徴量数、出力は１層目のノード数）を指定してインスタンス生成
linear = nn.Linear(64, 30)

In [3]:
# 生成したLayerのパラメタを確認
list(linear.parameters())

[Parameter containing:
 tensor([[-1.1046e-02,  5.5857e-03,  2.5025e-02,  ...,  8.1937e-03,
          -5.6370e-02, -1.0976e-01],
         [-7.9472e-02,  6.6317e-02,  1.1154e-01,  ...,  2.5045e-02,
           7.5435e-02,  5.9789e-02],
         [-8.0149e-02, -1.1696e-01, -1.0465e-01,  ...,  1.1494e-01,
           9.0277e-02, -1.0800e-01],
         ...,
         [-1.2304e-01,  5.6580e-05, -3.3194e-03,  ..., -2.8344e-02,
           5.6041e-02, -6.5417e-02],
         [ 4.2861e-03, -5.8977e-02,  1.1517e-01,  ...,  8.6463e-02,
          -8.2400e-02,  8.5337e-02],
         [ 4.0806e-02,  7.5585e-02, -1.6125e-02,  ..., -9.0116e-02,
          -1.9493e-02, -9.3259e-02]], requires_grad=True),
 Parameter containing:
 tensor([ 0.0726,  0.0846, -0.0918, -0.0318, -0.0923, -0.0496,  0.0174,  0.0853,
         -0.0805,  0.0751, -0.0512, -0.0234,  0.1060,  0.0289,  0.0721,  0.0784,
         -0.0364, -0.1085,  0.0797,  0.0177,  0.0273, -0.0026, -0.0316, -0.0169,
          0.0304,  0.1206, -0.0483, -0.0564, 

In [4]:
# 重みを確認
print(linear.weight)
print(linear.weight.shape)

Parameter containing:
tensor([[-1.1046e-02,  5.5857e-03,  2.5025e-02,  ...,  8.1937e-03,
         -5.6370e-02, -1.0976e-01],
        [-7.9472e-02,  6.6317e-02,  1.1154e-01,  ...,  2.5045e-02,
          7.5435e-02,  5.9789e-02],
        [-8.0149e-02, -1.1696e-01, -1.0465e-01,  ...,  1.1494e-01,
          9.0277e-02, -1.0800e-01],
        ...,
        [-1.2304e-01,  5.6580e-05, -3.3194e-03,  ..., -2.8344e-02,
          5.6041e-02, -6.5417e-02],
        [ 4.2861e-03, -5.8977e-02,  1.1517e-01,  ...,  8.6463e-02,
         -8.2400e-02,  8.5337e-02],
        [ 4.0806e-02,  7.5585e-02, -1.6125e-02,  ..., -9.0116e-02,
         -1.9493e-02, -9.3259e-02]], requires_grad=True)
torch.Size([30, 64])


確かにWのshapeが出力x入力になっている。

In [6]:
print(linear.bias)
print(linear.bias.shape)

Parameter containing:
tensor([ 0.0726,  0.0846, -0.0918, -0.0318, -0.0923, -0.0496,  0.0174,  0.0853,
        -0.0805,  0.0751, -0.0512, -0.0234,  0.1060,  0.0289,  0.0721,  0.0784,
        -0.0364, -0.1085,  0.0797,  0.0177,  0.0273, -0.0026, -0.0316, -0.0169,
         0.0304,  0.1206, -0.0483, -0.0564,  0.0212,  0.0984],
       requires_grad=True)
torch.Size([30])


In [13]:
X = torch.randn((5, 64)) # 64の特徴量を持つ5データを定義
# インスタンスにtensorを入れてcallすることで、forwardが計算される。
Z = linear(X)

In [14]:
print(Z)
print(Z.shape)

tensor([[-0.1680,  0.5441, -1.7527, -0.2023,  0.3643, -0.2527,  0.3505,  0.4083,
         -0.4886, -0.9790, -0.1874,  0.0556,  0.8950, -1.1019, -0.1446,  0.0945,
         -0.5899,  0.1869,  0.1683, -0.8990, -0.5504,  0.4096,  0.5353,  0.2216,
         -0.9816,  0.2632, -0.4898,  0.2971,  1.1802, -0.1012],
        [-0.5620,  0.9237,  0.3814, -1.2170, -0.2731, -0.1448,  0.1121, -0.4421,
         -0.4533, -0.2588, -0.2827, -0.6483,  0.5038, -0.1167,  0.2720,  0.4148,
         -0.5469,  0.0335,  0.0597, -0.1675, -0.1984, -0.8196, -0.4251,  0.2787,
          0.6631,  0.1476, -0.6547, -0.2453, -0.3012,  0.0060],
        [ 0.9574,  0.3612, -0.3115, -0.7415,  0.1366, -0.0830,  0.5199, -0.5133,
          0.1683,  0.6514, -0.1524,  0.4644,  0.2014,  0.4153,  0.2770, -0.3030,
          0.3074,  0.5833, -0.0500,  0.4514,  0.1932, -0.0686, -0.0691,  0.3298,
          0.4217,  0.5895, -0.0710,  0.4347, -0.5996,  0.0111],
        [ 0.2486,  0.8582, -1.4101,  0.6200,  0.2725, -0.0588,  0.3219, -0.1364

## nnモジュールでMLPモデルを作る -1
作成の仕方は大きく３つあり、ここでは1.について実装する。  
1. nn.<class>とF.<function>を組み合わせる  
    * nn.Moduleクラスを継承してMLPクラスを作成
    * パラメタを持つ層はnn.<class>で定義する。
    * パラメタを持たない操作（活性化関数など）はF.<functional>で定義する  
    →パラメタを持つ層の管理が楽というメリットがある。
2. nn.<class>のみで構成する。
3. nn.Sequentialを使う  


In [2]:
class MLP(nn.Module):
    def __init__(self, num_in, num_hidden, num_out):
        # 親クラスのinitを呼び出す。
        super().__init__()
        # 全結合層はパラメタを持つのでnn.で定義する
        self.linear_1 = nn.Linear(num_in, num_hidden)
        self.linear_2 = nn.Linear(num_hidden, num_out)

    # nn.Moduleにもforwardメソッドがあり、ここでオーバーライドしている。
    def forward(self, x):
        # デバッグ用
        # z1 = self.linear_1(z)
        # a1 = F.relu(z1)
        # z2 = self.linear_2(a1)
        # 隠れ層の線形変換 →ReLu適用→ 隠れ層の線形変換を一気にやっている。
        z = self.linear_2( F.relu( self.linear_1(x)) )
        return z

In [3]:
model = MLP(64, 30, 10)

In [8]:
test_tensor = torch.randn((1, 64))
# forwardの計算
model(test_tensor)

tensor([[-0.1167,  0.2524, -0.0680, -0.2275,  0.1086,  0.5086,  0.2032, -0.0763,
          0.2407,  0.5020]], grad_fn=<AddmmBackward0>)

### ※補足  
model()のような形で呼び出されるものは__call__()に実装するのがpythonの慣例。  
上記のMLPクラスでは特に__call__を修正していないのに、  
forwardが計算されたのは、親クラスのnn.Moduleにも__call__()があり、この中でforwardを呼び出す実装になっているため。  
つまり、__call__は継承元のまま使い、forwardはオーバーライドしているということ。
